In [ ]:
# Imports
import os
import subprocess
import sys

# REMOVE!!
from ir_utils import *

# model

from ir_lmart import *

# HPO

from hpo import *
from HpoWorker import *

import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH

# from hpbandster.core.worker import Worker

import logging
logging.basicConfig(level=logging.WARNING)

# HPO server and stuff

# import logging
# logging.basicConfig(level=logging.WARNING)

import argparse

import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres

from hpbandster.optimizers import BOHB as BOHB
from hpbandster.optimizers import RandomSearch as RS
from hpbandster.examples.commons import MyWorker
import random

# In[2]:


# In[3]:

In [ ]:
def tickets(max):
    return {i:i for i in range(1,max)}

In [ ]:
million_tickets = tickets(1000000)

In [ ]:
len(million_tickets)

In [ ]:
def get_train_budget_data_file(budget, query_list, train_data_file):
    # Budget is percentage of training data: 
    # min_budget = 10%
    # max_budget = 100%
    if (int(budget) <= 100 or int(budget) >= 10):
        len_queries = len(query_list)
        budgeted_queries = round(len_queries * (budget / 100))
        print('total budget:', len_queries)
        print('allocated budget:', budgeted_queries)
        train_budget_queries_file = train_data_file + '_budget' + str(budget)
        if not os.path.exists(train_budget_queries_file):
            with open(train_data_file, 'rt') as f_in:
                with open(train_budget_queries_file, 'wt') as budget_file_out:
                    for query_feature in f_in:                        
                        qid = query_feature.split()[1].split(':')[1]
#                         print(qid)
                        if qid in query_list[0:budgeted_queries]:
                            
                            budget_file_out.write(query_feature)
        else:
            print("File already exists")
            return train_budget_queries_file                
    else:
        print('Budget is outside the limits (10% < b < 100%): ', budget)
        return 

In [ ]:
# Functions
def generate_run_file(pre_run_file, run_file):
    
    with open(pre_run_file, 'rt') as input_f:
        pre_run = input_f.readlines()
#         print(type(pre_run))
    with open(run_file, 'wt') as out_f:
        for line in pre_run:
            out_f.write(line.replace('docid=','').replace('indri', 'lambdaMART'))
        
# In[4]:

In [ ]:
class fakeParser:
    def __init__(self):
        self.min_budget = 1
        self.max_budget = 100
        self.n_iterations = 20
        self.n_workers = 1
        self.hpo_method = 'rs'
        

In [ ]:
# Main
if __name__ == "__main__":
    
    # Options and variables
    
    parser = argparse.ArgumentParser(description='Example 1 - sequential and local execution.')
    parser.add_argument('--min_budget',   type=int, help='Minimum (percentage) budget used during the optimization.',    default=10)
    parser.add_argument('--max_budget',   type=int, help='Maximum (percentage) budget used during the optimization.',    default=100)
    parser.add_argument('--n_iterations', type=int,   help='Number of iterations performed by the optimizer', default=500)
    parser.add_argument('--n_workers', type=int,   help='Number of workers to run in parallel.', default=5)
    parser.add_argument('--dataset',   type=str, help='')
    parser.add_argument('--fold', type=str,   help='')
    parser.add_argument('--hpo_method',   type=str, help='')

#     args=parser.parse_args()
    args = fakeParser()
    
    
    hpo_method = args.hpo_method
#     dataset = sys.argv[1] # 'bioasq'
#     workdir = './' + dataset + '_dir/'
#     data_split = sys.argv[2] # 'test'

    dataset = 'bioasq'
    workdir = './' + dataset + '_dir/'
    data_split =  'train'
    k_fold = 's1' 
    ranklib_location = '../../../ranklib/'
    trec_eval_command = '../../eval/trec_eval'
    
    l2r_model = '_lmart_'
    confdir = './' + dataset + '_config/'
    enabled_features_file = confdir + dataset + l2r_model + 'enabled_features'
    
    metric2t = 'MAP' # 'MAP, NDCG@k, DCG@k, P@k, RR@k, ERR@k (default=ERR@10)'
    
    ranker_type = '6' # LambdaMART
    
    # normalization: Feature Engineering?
    norm_params = ['-norm', 'zscore'] # 'sum', 'zscore', 'linear'
    
    
    if hpo_method == 'rs':
        hpo_run_id = "RandomSearch"
        # Start a nameserver (see example_3)
        NS = hpns.NameServer(run_id = hpo_run_id, host='127.0.0.1', port=None)
        NS.start()
    elif hpo_method == 'bohb':
        hpo_run_id = "BOHB"
        NS = hpns.NameServer(run_id = hpo_run_id, host='127.0.0.1', port=None)
        NS.start()
    

In [ ]:
    if (not args.fold or args.dataset == 'bioasq'):
        args.fold = ['']
    elif args.fold == 'all':
        args.fold = ['1','2','3','4','5']
    #         args.fold = ['1']
    else:
        args.fold = [args.fold]


    for f in args.fold:

        fold = f # '1'

        if args.dataset == 'bioasq':
            fold_dir = workdir
            dataset_fold = dataset 
            train_queries_file = '../../bioasq_data/bioasq.' + 'train' + '.json'
        else:
            fold_dir = workdir + 's' + fold + '/'
            dataset_fold = dataset + '_' + fold
            train_queries_file = '../../robust04_data/split_' + fold + '/rob04.' +  'train' + '.s' + fold + '.json'
            

        train_data_file = fold_dir + dataset + '_train' + '_features'
        val_data_file = fold_dir + dataset + '_dev' + '_features'
        test_data_file = fold_dir + dataset + '_test' +  '_features'
        qrels_val_file = fold_dir + dataset + '_dev' + '_qrels'
        
        l2r_params = [
            '-validate',
            val_data_file,
            '-ranker',
            ranker_type,
            '-metric2t',
            metric2t,
            '-feature',
            enabled_features_file
        ]
        
        # Run train
        lmart_model = L2Ranker(ranklib_location, l2r_params, norm_params)
        

In [ ]:
        save_model_prefix = fold_dir + dataset_fold + l2r_model
        
        run_file_prefix = fold_dir + 'run_' + dataset_fold + l2r_model

        # Preparing budgeted train features data 
        query_list = load_queries(train_queries_file)
        qid_list = [q['id'] for q in query_list] 
        len_queries = len(qid_list) 
        real_max_budget = round((len_queries * args.max_budget) / 100)
        real_min_budget = round((len_queries * args.min_budget) / 100)

        # train_features_file =  fold_dir + dataset + '_' + 'train' + '_features'
        train_features_file =  fold_dir + dataset + '_' + 'train' + '_features'


        budget_train_features_file = train_data_file
        # budget_train_features_file = workdir + 'budget_ejemplo'

        workers=[]
        for i in range(args.n_workers):
            worker = HpoWorker(lmart_model, save_model_prefix, run_file_prefix, budget_train_features_file, qid_list, trec_eval_command, 
                               qrels_val_file, million_tickets, nameserver='127.0.0.1',run_id=hpo_run_id, id=i)
            worker.run(background=True)
            workers.append(worker)


        # In[13]:

In [ ]:
# # Run an optimizer (see example_2)


In [ ]:
# Random search

if hpo_method == 'rs':
    rs = RS(  configspace = worker.get_configspace(),
                          run_id = hpo_run_id, 
                          min_budget = real_max_budget, max_budget = real_max_budget
                   )
    res = rs.run(n_iterations = args.n_iterations, min_n_workers = args.n_workers)

    rs.shutdown(shutdown_workers=True)
elif hpo_method == 'bohb':
    bohb = BOHB(  configspace = worker.get_configspace(),
                          run_id = hpo_run_id, 
                          min_budget = real_min_budget, max_budget = real_max_budget
                   )
    res = bohb.run(n_iterations = args.n_iterations, min_n_workers = args.n_workers)
    bohb.shutdown(shutdown_workers=True)


# In[14]:


# Step 4: Shutdown
# After the optimizer run, we must shutdown the master and the nameserver.



In [ ]:

NS.shutdown()


# In[15]:


# Step 5: Analysis
# Each optimizer returns a hpbandster.core.result.Result object.
# It holds informations about the optimization run like the incumbent (=best) configuration.
# For further details about the Result object, see its documentation.
# Here we simply print out the best config and some statistics about the performed runs.
id2config = res.get_id2config_mapping()
incumbent = res.get_incumbent_id()


# In[16]:


all_runs = res.get_all_runs()



print('Best found configuration:', id2config[incumbent]['config'])
print('A total of %i unique configurations where sampled.' % len(id2config.keys()))
print('A total of %i runs where executed.' % len(res.get_all_runs()))
print('Total budget corresponds to %.1f full function evaluations.'%(sum([r.budget for r in all_runs])/args.max_budget))
print('The run took  %.1f seconds to complete.'%(all_runs[-1].time_stamps['finished'] - all_runs[0].time_stamps['started']))


# In[18]:


#     qrels_test_file = './bioasq_dir/bioasq.test_qrels'
#     run_val_file = './this.file'
#     lmart_model = res['info']
#     lmart_model.gen_run_file(test_data_file, run_val_file)
#     eval(trec_eval_command, qrels_test_file, run_val_file)


# In[10]:

In [ ]:
res.get_incumbent_id()

In [ ]:
id2config[res.get_incumbent_id()]

In [ ]:
# res.get_learning_curves()

In [ ]:
res.get_all_runs()

In [ ]:
res.get_id2config_mapping()